In [1]:
import json
import os

import pandas as pd
from pygments import highlight
from pygments.formatters import Terminal256Formatter
from pygments.lexers import PythonLexer
from pygments.styles import get_style_by_name


# def read_dir_data(base_path):
#     data = []
#     for dir_name in os.listdir(base_path):
#         dir_path = os.path.join(base_path, dir_name)
#         if os.path.isdir(dir_path):
#             record = {"directory": dir_name}
#             with open(os.path.join(dir_path, "metadata.json"), "r") as file:
#                 metadata = json.load(file)
#             record.update(metadata)
#             with open(os.path.join(dir_path, "question.txt"), "r") as file:
#                 record["question"] = file.read().strip()
#             solutions_path = os.path.join(dir_path, "solutions.json")
#             if os.path.isfile(solutions_path):
#                 with open(solutions_path, "r") as file:
#                     record["solutions"] = json.load(file)
#             data.append(record)
#     df = pd.DataFrame(data)
#     return df


# base_path = "train"
# df = read_dir_data(base_path)

In [30]:
# save the df as a pickle file
df.to_pickle("train.pkl")

# tell me size in MB
print("Size of the dataframe in MB:", df.memory_usage(deep=True).sum() / 1024 ** 2)

Size of the dataframe in MB: 14.622417449951172


In [23]:
import asyncio

from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.runnables.base import RunnableBinding
from langchain_openai import ChatOpenAI


def get_llm(model: str):
    if model == "cleverchain-gpt-4o":
        openai_gpt4_args = {
            "seed": 123,
            # "function_call": None,
            # "response_format": {"type": "json_object"},
        }
        return ChatOpenAI(
            model="gpt-4o-2024-05-13",
            temperature=0.5,
            max_tokens=2000,
            model_kwargs=openai_gpt4_args,
            openai_api_key='put your openai key here'
        )




# Asynchronous function to process each problem description
async def process_description(question, solution):
    sys_msg = """
    Given a competitive programming problem description and its expert solution, simplify the description to focus strictly on computational and data structure terms. Remove any thematic or domain-specific language, abstracting the problem to its algorithmic essentials.

    Task:
    - Parse the input problem description to identify core computational processes and data handling.
    - Remove all thematic or narrative elements, replacing them with general computational terms that capture the essence of the problem without domain-specific context.
    - Output a streamlined version that uses only terms relevant to computer science, data structures, and algorithms, ensuring it is suitable for database indexing and algorithm lookup.


    Example:
    Input:
    Problem Description: 'We have a sandglass consisting of two bulbs, bulb A and bulb B. These bulbs contain some amount of sand. When we put the sandglass, either bulb A or B lies on top of the other and becomes the upper bulb. The other bulb becomes the lower bulb. The sand drops from the upper bulb to the lower bulb at a rate of 1 gram per second. When the upper bulb no longer contains any sand, nothing happens. Initially at time 0, bulb A is the upper bulb and contains a grams of sand; bulb B contains X-a grams of sand (for a total of X grams). We will turn over the sandglass at time r_1, r_2,..,r_K. Assume that this is an instantaneous action and takes no time. Here, time t refers to the time t seconds after time 0. You are given Q queries. Each query is in the form of (t_i,a_i). For each query, assume that a=a_i and find the amount of sand that would be contained in bulb A at time t_i.'

    Solution: Simplified version focusing on the essential algorithmic steps.

    Output:
    ```
    'Given two variables, A and B, holding a total of X units. Initially, A has 'a' units and B has 'X-a' units. Units transfer from the active variable to the passive variable at a rate of 1 unit per second. Switches between which variable is active occur at times r_1, r_2, ..., r_K. Answer Q queries, each specifying a time t_i and an initial amount 'a_i' for A. Determine the number of units in A at each specified time t_i.

    key DSA terms: "BFS, state transition......"
    ```

    This task will help in creating clear and direct problem statements for use in vector databases and algorithm lookup systems.

    Do not include input and output formats in the simplified version. Only focus on the core computational and data handling aspects of the problem. Ensure that the simplified version is concise and free of thematic or narrative elements. The output should be suitable for indexing in a vector database and RAG lookup system.

    Once again, it is very very important that you exclude jargon. Topical terms, proper nouns, hypotehtical names, prooblem lore, cannon, and other weird noise should always be removed. The goal is to make the problem statement as generic as possible, so that it can be used in a variety of contexts. This is a very important task, so please take your time and do it well.
    """
    system_message = SystemMessage(content=sys_msg)

    task = f"""
    ---

    This is the first competition problem, TASK 1:
    ```
    {question}
    ```

    This is the provided expert solution:
    ```
    {solution}
    ```
    """
    initial_human_message = HumanMessage(content=task)
    messages = [system_message, initial_human_message]
    return await LLM.ainvoke(messages)

LLM = get_llm("cleverchain-gpt-4o")

demo_testing = df[df["difficulty"] == "competition"][23:26]
questions = demo_testing["question"]
solutions = demo_testing["solutions"]


In [24]:
print(questions.iloc[1])

There are $n$ persons who initially don't know each other. On each morning, two of them, who were not friends before, become friends.

We want to plan a trip for every evening of $m$ days. On each trip, you have to select a group of people that will go on the trip. For every person, one of the following should hold:   Either this person does not go on the trip,  Or at least $k$ of his friends also go on the trip. 

Note that the friendship is not transitive. That is, if $a$ and $b$ are friends and $b$ and $c$ are friends, it does not necessarily imply that $a$ and $c$ are friends.

For each day, find the maximum number of people that can go on the trip on that day.


-----Input-----

The first line contains three integers $n$, $m$, and $k$ ($2 \leq n \leq 2 \cdot 10^5, 1 \leq m \leq 2 \cdot 10^5$, $1 \le k < n$) — the number of people, the number of days and the number of friends each person on the trip should have in the group.

The $i$-th ($1 \leq i \leq m$) of the next $m$ lines con

In [26]:
solutions

303    [import sys\n\nn, m = list(map(int, sys.stdin....
310    [from collections import deque\n\ndef solve(ad...
321    [q = int(input())\n\n\n\ndef full_way(u):\n   ...
Name: solutions, dtype: object

In [27]:
tasks = []
for q, s in zip(questions, solutions):
    q = q.strip()
    s = s[0]
    print("-*" * 40)
    print("-*" * 40)
    print("-*" * 40)
    print(q)
    print("-*" * 40)
    print(highlight(s, PythonLexer(), Terminal256Formatter(style='monokai')))
    print("-*" * 40)
    print("-*" * 40)
    print("-*" * 40)
    tasks.append(process_description(q, s))

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
Leha plays a computer game, where is on each level is given a connected graph with n vertices and m edges. Graph can contain multiple edges, but can not contain self loops. Each vertex has an integer d_{i}, which can be equal to 0, 1 or  - 1. To pass the level, he needs to find a «good» subset of edges of the graph or say, that it doesn't exist. Subset is called «good», if by by leaving only edges from this subset in the original graph, we obtain the following: for every vertex i, d_{i} =  - 1 or it's degree modulo 2 is equal to d_{i}. Leha wants to pass the game as soon as possible and ask you to help him. In case of multiple correct answers, print any of them.


-----Input-----

The first line contains two integers n, m (1 ≤ n ≤ 3·10^5, n - 1 ≤ 

In [28]:
results = await asyncio.gather(*tasks)

In [29]:
for res in results:
    print(res.content)
    print("-*" * 40)

Simplified Problem Description:
```
Given a connected undirected graph with \( n \) vertices and \( m \) edges, where each vertex has an associated integer \( d_i \) that can be 0, 1, or -1. The graph may have multiple edges but no self-loops. The task is to find a subset of edges such that, after removing the other edges, the degree of each vertex \( i \) is either -1 or its degree modulo 2 equals \( d_i \). If such a subset exists, output it; otherwise, output -1.

Input:
- Two integers \( n \) and \( m \) representing the number of vertices and edges.
- A list of \( n \) integers representing \( d_1, d_2, ..., d_n \).
- \( m \) pairs of integers representing the edges.

Output:
- If a valid subset exists, output the number of edges in the subset followed by the indices of these edges.
- If no valid subset exists, output -1.

Example:
Input:
4 5
0 0 0 -1
1 2
2 3
3 4
1 4
2 4

Output:
0
```

Key DSA terms: graph, connected graph, vertices, edges, degree, subset, modulo, BFS, DFS, state